In [12]:
from nonTracking import *
#=============================================================================
# Functions
#=============================================================================
def simulate_BM(L, rho, D, dt, p_off, mu_on, nF):
    '''Generates sequence of distance matrices between all points in each pair 
    of succesive frames as well as the distances corresponding to the true
    matchings.'''
#    positions=[] # Positions in each frame
    N=avg_N
    drs_ij=[]
    drs_true=[]
    Ns=[N]
    Ns_true=[]
    for frame in range(nF):
        x=array([L*rand(N)-L/2.,L*rand(N)-L/2]) # Initialize positions inside square
        dr_true=reshape(sqrt(2*D*dt)*randn(2*N),[2,N]) # Particle displacements
        y_true=x+dr_true # Positions in subsequent frame
        #--- Simulate Blinking: ------------------------------------------------------
        # Particles disappearing:
        i_off=[]
        for i in range(N):
            if rand()<p_off:
                i_off.append(i)
        n_off=len(i_off)
        dr_true=array([np.delete(dr_true[0],i_off),np.delete(dr_true[1],i_off)])
        drs_true.append(dr_true)
        # Particles appearing:
        m_on=np.random.poisson(mu_on)
        y_new=array([L*rand(m_on)-L/2.,L*rand(m_on)-L/2])+reshape(sqrt(2*D*dt)*randn(2*m_on),[2,m_on])
        #--- Calculate observed positions: -------------------------------------------
        y=array([np.append(np.delete(y_true[0],i_off),y_new[0]),np.append(np.delete(y_true[1],i_off),y_new[1])])
        N_true=N-n_off
        Ns_true.append(N_true)
        M=N_true+m_on
        Delta=M-N
        # True assignement matrix:
        pi_ij_true=zeros([N,M])
        for i_,j_ in zip(np.delete(arange(N),i_off),range(N_true)):
            pi_ij_true[i_,j_]=1.
        mpa_row_true=np.delete(arange(N),i_off)
        mpa_col_true=np.arange(N_true)
        # Matrix of distances:
        dr_ij=distance_matrices(x,y)
        drs_ij.append(dr_ij)
        # Update for next frame and add data to arrays:
        N=M
        Ns.append(N)
    return (Ns_true, drs_true, Ns, drs_ij) 


In [33]:
#=============================================================================
# Parameters
#=============================================================================
# Initialize seed of PRNG:
seed(129041982)
#-----------------------------------------------------------------------------
# Physical parameters:
#-----------------------------------------------------------------------------
L=sqrt(10.) # Linear size of box [mu]
rho=1. # Particle density [1/mu^2]
avg_N=int(rho*L**2) # Average number of particles in each frame
D=1. # Diffusion coefficient [mu^2/s]
dt=0.01 # Time lapse between images [s]
p_off=0.25 # Probability for each particle to disappear per frame
mu_off=avg_N*p_off # Average number of particles disappearing per frame
mu_on=mu_off # Average number of particles appearing per frame (set equal to mu_off)
nF=100 # Number of time lapses recorded (= number of frames minus one)
ensembleSize=10 # Size of Monte Carlo ensemble
tries=1 # Number of 
nL=0.0001
#-----------------------------------------------------------------------------
# Fit parameters:
#-----------------------------------------------------------------------------
Da=0.1*D
Db=10.*D

t_tot=time()
print "Parameter values: L =",L, "; rho =",rho, "; D =",D, "; dt =",dt, "; p_off =",p_off, "; mu_on +", mu_on, "; number of frames:",nF

# Generate trajectories:
Ns_true, drs_true, Ns, drs_ij = simulate_BM(L, rho, D, dt, p_off, mu_on, nF)
print "\Average number of particles per frame: "+str(np.average(Ns))
print "Average number of true links per frame: "+str(np.average(Ns_true))

print "\nFit using:"
# True assignment:
t_true=time()
sol_true=np.sum([np.average(dr_**2)*len(dr_)/(2.*dt) for dr_ in drs_true])/np.sum([len(dr_) for dr_ in drs_true])
print "  --- true matching: ---"
print "    D_est =", sol_true, "(time to fit: "+str(time()-t_true)+"s )" 

# Sum-product BP:
t_BP=time()
sol_BP=optim.minimize_scalar(marginal_minusLogLikelihood_multiframe,bounds=[Da,Db],\
                             args=(dt,drs_ij,gamma,epsilon,max_it,L,mu_off,mu_on,Ns,tries,nL),method='bounded')
print "  --- non-tracking (BP): ---"
print "    D_est =", sol_BP.x, "(time to fit: "+str(time()-t_BP)+"s )" 

# MPA:
t_MPA=time()
sol_MP=optim.minimize_scalar(MPA_minusLogLikelihood_multiframe,bounds=[Da,Db],\
                             args=(dt,drs_ij,L,mu_off,mu_on,Ns),method='bounded')
print " --- most probable assignment: ---"
print "    D_est =", sol_MP.x, "(time to fit: "+str(time()-t_MPA)+"s )" 


Parameter values: L = 3.16227766017 ; rho = 1.0 ; D = 1.0 ; dt = 0.01 ; p_off = 0.25 ; mu_on + 2.5 ; number of frames: 100
\Average number of particles per frame: 10.7128712871
Average number of true links per frame: 8.03

Fit using:
  --- true matching: ---
    D_est = 1.05358162347 (time to fit: 0.00217008590698s )
  --- non-tracking (BP): ---
    D_est = 0.9586699148 (time to fit: 28.7526910305s )
 --- most probable assignment: ---
    D_est = 1.11184296648 (time to fit: 21.2193648815s )
